In [1]:
# coding: utf-8
import re 
import pandas as pd
import requests
import random
import json
from urllib import parse
from urllib import request
import time
from datetime import datetime
from datetime import timedelta
header={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36',
    'Connection':'keep-alive'
}
cookies = '__mta=147569288.1539613293789.1539613310187.1539616722700.3; v=3; \
        _lxsdk_cuid=166781a789ec8-0139751cb2ee6e-8383268-144000-166781a789fc8; \
        __mta=147569288.1539613293789.1539613293789.1539613310187.2; uuid_n_v=v1; \
        iuuid=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; webp=true; ci=96%2C%E6%B5%8E%E5%8D%97; \
        _lxsdk=9AF20AC0D08D11E89D508D0D63C93F71D10FE6A2B71B406D800371E2CD9160FE; _lxsdk_s=166784ecee0-b1a-f0b-22%7C%7C2'
cookie = {}
for line in cookies.split(';'):
    name, value = cookies.strip().split('=', 1)
    cookie[name] = value

In [2]:
def get_id_by_name(name):
    url = 'http://maoyan.com/query?'
    dict1 ={'kw': name}
    url_data = parse.urlencode(dict1) #unlencode()将字典{k1:v1,k2:v2}转化为k1=v1&k2=v2
    html = requests.get(url=url+url_data, cookies=cookie,headers=header).content.decode('utf-8')
    movie_item = re.findall(r'data-val=\"{movieid:(.*?)}">', html)
    if len(movie_item)==0:
        return -1
    else:
        return movie_item[0]

In [42]:
def get_release_time_by_name(name):
    id = get_id_by_name(name)
    if id == -1:
        return None
    else:
        url = 'http://maoyan.com/films/'+get_id_by_name(name)
#         print(url)
        html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
        s = r'<li class=\"ellipsis\">(.*?)映<\/li>'
        release_time = re.findall(s, html)
        return release_time[0][0:10]#只截取时间 如 2018-09-30 

In [66]:
def get_comments_all_test(name):
    comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])
    start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # 获取当前时间，从当前时间向前获取
    release_time = str(get_release_time_by_name(name))
    end_time =  release_time+' 00:00:00'
    comments.to_csv('all_'+name+'.csv',index=False,encoding="utf_8_sig")
    count=0
    pages =15
#     for i in range(0,20):
    while pages ==15:  
        try:
            time.sleep(1)
            
            url = 'http://m.maoyan.com/mmdb/comments/movie/'+get_id_by_name(name)+'.json?_v_=yes&offset=0&startTime=' + start_time.replace(' ', '%20')
            print(str(count)+"  "+url)
            count = count+1
            html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
            data = json.loads(html)['cmts']
            pages=len(data)

            for item in data:
                if item.get('cityName')==None or item.get('time')==None or item.get('score')==None or item.get('content')==None or item.get('nick')==None:
                    print("jump")
                else:  
                    try:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':item['gender']},
                                                       ignore_index=True)
                    except:
                        comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
                                                        'score':str(item['score']),'comment':item['content'],
                                                    'nick':item['nick'],'gender':0},
                                                       ignore_index=True)
            comments.to_csv('all_'+name+'.csv',index=False,mode='a', header=False,encoding="utf_8_sig")
            
            start_time = data[14]['startTime']  # 获得末尾评论的时间
            start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=-1)  # 转换为datetime类型，减1秒，避免获取到重复数据
            start_time = datetime.strftime(start_time, '%Y-%m-%d %H:%M:%S')  # 转换为str
            
            comments.drop(comments.index,inplace=True)#清空comments   

        except Exception as e:
            time.sleep(0.5)
            print(e)
            continue
    print(end_time)

In [67]:
get_comments_all_test("嘻哈英熊")

0  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-18%2019:32:28
1  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-16%2017:26:33
2  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-14%2012:54:28
3  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-13%2000:30:11
4  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-10%2011:24:38
5  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-08%2012:48:33
6  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-07%2018:07:08
7  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-07%2014:40:02
8  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-07%2011:14:50
9  http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&o

In [46]:
get_release_time_by_name("嘻哈英熊")

'2018-10-04'

In [12]:
 url = 'http://m.maoyan.com/mmdb/comments/movie/672137.json?_v_=yes&offset=0&startTime=2018-10-05%2018:44:06'

html = requests.get(url=url, cookies=cookie,headers=header).content.decode('utf-8')
data = json.loads(html)['cmts']

In [14]:
 comments = pd.DataFrame(columns=['data','score','city','comment','nick','gender'])

In [25]:
i=0
# try:
for item in data:
    i=i+1
    print(item.get('cityName')==None or True)
#         try:
#             comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
#                                             'score':str(item['score']),'comment':item['content'],
#                                         'nick':item['nick'],'gender':item['gender']},
#                                            ignore_index=True)
#         except:
#             comments = comments.append({'data':item['time'].split(' ')[0],'city':item['cityName'],
#                                             'score':str(item['score']),'comment':item['content'],
#                                         'nick':item['nick'],'gender':0},
#                                            ignore_index=True)
# except Exception as e:
#     print(e)
#     print(i)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
